In [1]:
from gensim.models import Word2Vec
import annoy # fast approximate KNN
import numpy as np

In [2]:
!head ./UP000005640.goByGeneC

"V1" "V2"
"A0PJZ0" "MKLFGFRSRRGQTVLGSIDHLYTGSGYRIRYSELQKIHKAAVKGDAAEMERCLARRSGDLDALDKQHRTALHLACASGHVKVVTLLVNRKCQIDIYDKENRTPLIQAVHCQEEACAVILLEHGANPNLKDIYGNTALHYAVYSESTSLAEKLLFHGENIEALDKV" ""
"P30461" "MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRFITVGYVDDTQFVRFDSDATSPRMAPRAPWIEQEGPEYWDRETQISKTNTQTYRENLRTALRYYNQSEAGSHTWQTMYGCDLGPDGRLLRGHNQLAYDGKDYIALNEDLSSWTAADTAAQITQLKWEAARVAEQLRAYLEGECVEWLRRYLENGKETLQRADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWEPSSQSTVPIVGIVAGLAVLAVVVIGAVVAAVMCRRKSSGGKGGSYSQAACSDSAQGSDVSLTA" ""
"P18825" "MASPALAAALAVAAAAGPNASGAGERGSGGVANASGASWGPPRGQYSAGAVAGLAAVVGFLIVFTVVGNVLVVIAVLTSRALRAPQNLFLVSLASADILVATLVMPFSLANELMAYWYFGQVWCGVYLALDVLFCTSSIVHLCAISLDRYWSVTQAVEYNLKRTPRRVKATIVAVWLISAVISFPPLVSLYRQPDGAAYPQCGLNDETWYILSSCIGSFFAPCLIMGLVYARIYRVAKLRTRTLSEKRAPVGPDGASPTTENGLGAAAGAGENGHCAPPPADVEPDESSAAAERRRRRGALRRGGRRRAGAEGGAGGADGQGAGPGAAESGALTASRSPGPGGRLSRASSRSVEFFLSRRRRARSSVCRRKVAQAREKRFTFVLAVVMGVFVLCWFPFFFSYSLYGICREACQVPGPLFKFFF

In [4]:
good_labels = set(['GO:0005634',# nucleus
'GO:0005576',# extracellular
'GO:0005829',# cytosol
'GO:0005856',# cytoskeleton
'GO:0005739',# mitochondrion
'GO:0005886',# cell_membrane
'GO:0005783',# endoplasmic_reticulum
'GO:0005794',# golgi_apparatus
'GO:0005773',# vacuole
'GO:0005777',# peroxysome
              ])

In [7]:
X = [] # sequence strings
Y = [] # list of labels for each sequence
all_y = set() # all the labels collected from the dataset
with open('./UP000005640.goByGeneC') as inp:
    inp.readline() # header
    for line in inp:
        tokens = map(lambda x: x.strip('"'), line.split(' ',2))
        
        object_labels = [t.strip().strip('"') for t in tokens[2].split(',') if len(t.strip())>0 and t.strip]
        object_labels = [y for y in object_labels if y in good_labels]
        if len(object_labels)>=1:
            X.append(tokens[1])
            Y.append(object_labels)
            all_y = all_y.union(set(Y[-1]))

In [8]:
print 'example of X', X[:3]
print 'example of Y', Y[:3]
print 'example of set of labels', list(all_y)[:3]
print 'total sequences', len(X)
print 'total labels', len(all_y)

example of X ['MASPALAAALAVAAAAGPNASGAGERGSGGVANASGASWGPPRGQYSAGAVAGLAAVVGFLIVFTVVGNVLVVIAVLTSRALRAPQNLFLVSLASADILVATLVMPFSLANELMAYWYFGQVWCGVYLALDVLFCTSSIVHLCAISLDRYWSVTQAVEYNLKRTPRRVKATIVAVWLISAVISFPPLVSLYRQPDGAAYPQCGLNDETWYILSSCIGSFFAPCLIMGLVYARIYRVAKLRTRTLSEKRAPVGPDGASPTTENGLGAAAGAGENGHCAPPPADVEPDESSAAAERRRRRGALRRGGRRRAGAEGGAGGADGQGAGPGAAESGALTASRSPGPGGRLSRASSRSVEFFLSRRRRARSSVCRRKVAQAREKRFTFVLAVVMGVFVLCWFPFFFSYSLYGICREACQVPGPLFKFFFWIGYCNSSLNPVIYTVFNQDFRRSFKHILFRRRRRGFRQ', 'MLVDGPSERPALCFLLLAVAMSFFGSALSIDETRAHLLLKEKMMRLGGRLVLNTKEELANERLMTLKIAEMKEAMRTLIFPPSMHFFQAKHLIERSQVFNILRMMPKGAALHLHDIGIVTMDWLVRNVTYRPHCHICFTPRGIMQFRFAHPTPRPSEKCSKWILLEDYRKRVQNVTEFDDSLLRNFTLVTQHPEVIYTNQNVVWSKFETIFFTISGLIHYAPVFRDYVFRSMQEFYEDNVLYMEIRARLLPVYELSGEHHDEEWSVKTYQEVAQKFVETHPEFIGIKIIYSDHRSKDVAVIAESIRMAMGLRIKFPTVVAGFDLVGHEDTGHSLHDYKEALMIPAKDGVKLPYFFHAGETDWQGTSIDRNILDALMLNTTRIGHGFALSKHPAVRTYSWKKDIPIEVCPISNQVLKLVSDLRNHPVATLMATGHPMVISSDDPAMFGAKGLSYDFYEVFMGIGGMKADLRTLKQLAMNSIKYSTLLESEKNTFMEIWKKRWDKFIADVATK', 'MGSP

In [9]:
def cv_iter(w2v_params):    
    """
    one ieration of cross-validation
    w2v_params is a dictionary with parameters of gensim.Word2Vec
    """
    indices = range(len(X))
    np.random.shuffle(indices)

    Xtrain = np.array(X)[indices[:len(indices)/2]]
    Ytrain = np.array(Y)[indices[:len(indices)/2]]
    Xtest = np.array(X)[indices[len(indices)/2:]]
    Ytest = np.array(Y)[indices[len(indices)/2:]]
    # split all the strings into lists for word2vec traing
    Xtrain = [list(x) for x in Xtrain]
    Xtest = [list(x) for x in Xtest]
    model = Word2Vec(Xtrain, **w2v_params)
    index = annoy.AnnoyIndex(w2v_params.get('size', 100)) # default value is 100 for gensim.Word2Vec
    for i in range(len(Xtrain)):
        v= np.mean([model[c] for c in Xtrain[i]], axis=0) # centroid
        index.add_item(i, v)
    index.build(100) # 100 is a number of trees in index, we can alibrate this value
    predicted = []
    for i in range(len(Xtest)):
        v= np.mean([model[c] for c in Xtest[i]], axis=0)
        id = index.get_nns_by_vector(v,1)[0] #  finding 1 nearest neighbour
        predicted.append(id)
    tp = 0
    fp = 0
    true_label_num = 0
    for i in xrange(len(Xtest)):
        true = set(Ytest[i])
        true_label_num+=len(true)
        pred = set(Ytrain[predicted[i]])
        tp += len(true&pred)
        fp += len(pred) - len(true&pred)

    prec=tp*1.0/(tp+fp)
    rec = tp*1.0/true_label_num
    return prec, rec,  2 * prec * rec/ (prec+rec)
    

In [10]:
def cv(w2v_params = {}, cv_num = 5 ):
    """
    Cross-validation function.
    Returns list with precision, recall and F1 values
    """
    precs, recs, fs = [],[],[]
    for _ in range(cv_num):
        p,r,f = cv_iter(w2v_params)
        precs.append(p)
        recs.append(r)
        fs.append(f)
    return precs, recs, fs

In [11]:
p,r,f = cv()
np.mean(p), np.mean(r), np.mean(f)

(0.4446942686542964, 0.45321409664909124, 0.44891123153915224)

In [13]:
p,r,f = cv(w2v_params={'size':5})
np.mean(p), np.mean(r), np.mean(f)

(0.39638578831190696, 0.392533077057106, 0.3944485865878974)